In [1]:
import architecture
import data_setup
import engine
import os 
import torch

c:\Users\JOinme\Downloads\GithubClone\TI_Final\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#  Set up data and folder zone

In [2]:
# # Set up folder and extract files in zip
# data_setup.extractzip()

In [3]:
# data_setup.concat_files("train")

In [4]:
# data_setup.concat_files("test")

In [5]:
# # Read and change data into DataLoader
# train_dataloader = data_setup.create_dataloaders(os.path.join("..", "Data", "train", "FLC_data.csv"), 
#                                                  target_col='close', 
#                                                  batch_size=64)


In [6]:
# next(iter(train_dataloader))[0].shape

In [7]:
# # Initialise model
# model = architecture.SPP(input_size=12,
#                          hidden_size=64,
#                          output_shape=1)

In [8]:
# # Loss function and optimizer
# loss_func = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(params=model.parameters(),
#                              lr=0.01)

In [9]:
# from torchinfo import summary

# summary(model,
#         input_size=(64, 12),
#         col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
#         row_settings = ["var_names"])

In [10]:
# writer = engine.create_writer(experiment_name="SPP",
#                               model_name="base")

In [11]:
# # Training
# results = engine.train(model=model,
#                        train_dataloader=train_dataloader,
#                        loss_func=loss_func,
#                        epochs = 5,
#                        optimizer=optimizer)
#                     #    writer=writer)

# Training

In [12]:
# import pandas as pd
# data = pd.read_csv(os.path.join("..", "Data", "train", "StockData_train.csv"))
# samples = data.sample(n=50000, ignore_index=True).drop('Unnamed: 0', axis = 1)
# samples.to_csv(os.path.join("..", "Data", "train", "StockData50_train.csv"))

In [13]:
lr = 0.0025
hidden_size = 256
batch_size = 512
model_name = "bnlstmfive32" + "_" + str(hidden_size) + "_" + str(batch_size) + "_" + str(lr)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [15]:
# Read and change data into DataLoader
train_dataloader = data_setup.create_dataloaders(train_path=os.path.join("..", "Data", "train", "StockData32_train.csv"), 
                                                 target_col='close', 
                                                 batch_size=batch_size)


In [16]:
# Initialise model
model = architecture.SPP(input_size=12,
                         hidden_size=hidden_size,
                         output_shape=1).to(device)

In [17]:
# Loss function and optimizer
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=lr)

In [18]:
from torchinfo import summary

summary(model, 
        input_size=(256, 12),
        col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
        row_settings = ["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
SPP (SPP)                                [256, 12]                 [256, 1]                  --                        True
├─BNLSTM (bnlstm1)                       [256, 12]                 [256, 256]                512                       True
│    └─BNLSTMCell (bnlstmcell)           --                        [256, 256]                275,456                   True
│    │    └─BatchNorm1d (bn_hidden)      [256, 1024]               [256, 1024]               2,048                     True
│    │    └─BatchNorm1d (bn_input)       [256, 1024]               [256, 1024]               2,048                     True
│    │    └─BatchNorm1d (bn_c)           [256, 256]                [256, 256]                512                       True
├─Dropout (dropout1)                     [256, 256]                [256, 256]                --                        --
├─BNL

In [19]:
writer = engine.create_writer(experiment_name="development",
                              model_name=model_name)

[INFO] An instane of SummaryWriter is created, saving to: ..\log\2025-03-17\development\bnlstmfive32_256_512_0.0025


In [ ]:
# Training
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       loss_func=loss_func,
                       epochs = 50,
                       optimizer=optimizer,
                       writer=writer,
                       device=device)

  2%|▏         | 1/50 [01:42<1:23:37, 102.39s/it]

Epoch: 1 | Train loss: 5.2216 | Train evaluation score: 2.2851


  4%|▍         | 2/50 [03:29<1:24:00, 105.02s/it]

Epoch: 2 | Train loss: 2.1783 | Train evaluation score: 1.4759


  6%|▌         | 3/50 [05:23<1:25:25, 109.06s/it]

...

Epoch: 10 | Train loss: 0.1525 | Train evaluation score: 0.3905


 22%|██▏       | 11/50 [23:18<1:22:04, 126.27s/it]

Epoch: 11 | Train loss: 0.1415 | Train evaluation score: 0.3762


 24%|██▍       | 12/50 [25:44<1:23:43, 132.19s/it]

Epoch: 12 | Train loss: 0.1139 | Train evaluation score: 0.3375


In [ ]:
engine.save_model(model=model,
                  model_name=model_name,
                  save_folder=os.path.join("..", "model"))

[INFO] Model is saved successfully to ..\model\bnlstmfive32_256_256_0005


'..\\model\\bnlstmfive32_256_256_0005'

In [ ]:
import pandas as pd
tosave = pd.DataFrame(results).T
tosave.columns = ['Loss', 'Eval_score']
tosave.to_csv(os.path.join("..", "log_df", model_name + ".csv"))